# Use PyTorch to predict handwritten digits
<table style="border: none" align="left">
   <tr style="border: none">
       <td style="border: none"><img src="https://github.com/IBM/pytorch-on-watson-studio/raw/master/doc/source/images/pytorch-pattern-header.jpg" width="600" alt="Icon"></td>
   </tr>
</table>

The sample model used here is the <a href="https://github.com/pytorch/examples/tree/master/mnist">MNIST model</a> from the official PyTorch examples repository. 

## Training MNIST Model

** PyTorch Tools & Libraries **

An active community of researchers and developers have built a rich
ecosystem of tools and libraries for extending PyTorch and supporting
development in areas from computer vision to reinforcement learning.

PyTorch's <a href="https://github.com/pytorch/vision" target="_blank"
rel="noopener no referrer">torchvision</a> is one of those packages.
`torchvision` consists of popular datasets, model architectures, and common
image transformations for computer vision.

This tutorial will use `torchvision's MNIST dataset` package to download
and process the training data.

The following code will download and process the MNIST training and test data. 

In [1]:
!pip install torchvision

In [2]:
import torch
from torchvision import datasets, transforms

data_dir = './data'

train_ds = datasets.MNIST(data_dir, train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
test_ds = datasets.MNIST(data_dir, train=False,
                         transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

0it [00:00, ?it/s]

9920512it [00:02, 4214465.69it/s]                             


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 139601.80it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:00, 3615810.28it/s]                           
8192it [00:00, 72302.50it/s]            


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


Have a look at the list of the created buckets and their contents.

** Define model **


In [3]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

** Define training loop **

In [4]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

** Test loop **

In [5]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

** Argument setting **

In [6]:
import argparse
    # Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
    
parser.add_argument('--save-model', action='store_true', default=False,
                    help='For Saving the current Model')
#args = parser.parse_args(['--epochs','1', '--no-cuda'])
args = parser.parse_args(['--epochs','1'])
print(args)

Namespace(batch_size=64, epochs=1, log_interval=10, lr=0.01, momentum=0.5, no_cuda=False, save_model=False, seed=1, test_batch_size=1000)


** Prepare for training **

In [7]:
import torch.optim as optim

use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(train_ds,
                        batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_ds, 
                        batch_size=args.test_batch_size, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

** Run the training loop **

In [8]:
for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.330959
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.355268
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.282203
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.262905
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.206378
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.234866
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.209639
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.159384
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.115429
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.010443
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.864485
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.780141
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.737139
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.590657
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.421334
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.512587
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.263209
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.198920
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.311268
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.036210
Train Epoch: 1 [12800/60000 (

** Save model **

In [9]:
args.save_model = True
saved_model_filename = "mnist_cnn.pt"
if (args.save_model):
    torch.save(model.state_dict(), saved_model_filename)

In [ ]:
!ls -o

** Work with the trained models **

Instantiate and load previously trained model parameters.

In [10]:
mnist_model = Net()
mnist_model.load_state_dict(torch.load(saved_model_filename, map_location='cpu'))

<All keys matched successfully>

In [11]:
!pip install wget

In [12]:
import os
import wget

images =[]
for i in range(1,10):
    filename = "img_"+str(i)+".jpg"
    images.append(filename)
    if not os.path.isfile(filename):
        path = "https://github.com/IBM/pytorch-on-watson-studio/raw/master/data/images/"+filename
        wget.download(path)

print(images)

['img_1.jpg', 'img_2.jpg', 'img_3.jpg', 'img_4.jpg', 'img_5.jpg', 'img_6.jpg', 'img_7.jpg', 'img_8.jpg', 'img_9.jpg']


Using the trained model to predict the digits in the sample image files.

In [13]:
import numpy as np
from IPython.display import display
from PIL import Image

**Python-First**

PyTorch is not a Python binding into a monolithic C++ framework.
It’s built to be deeply integrated into Python so it can
be used with popular Python libraries.

The code below shows how to convert a NumPy array to
a PyTorch tensor using `torch.from_numpy`.

In [14]:
digits = [i for i in range(10)]

mnist_model.eval()
for i, filename in enumerate(images):
    img = Image.open(filename).resize((28, 28)).convert('L')
    display(img)
    data = torch.from_numpy(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :])
    output = mnist_model(data)
    # get the index of the max log-probability
    prediction = output.max(1, keepdim=True)[1]
    print("Prediction for image number", i+1, "is:",  digits[prediction[0,0]])

Prediction for image number 1 is: 2


Prediction for image number 2 is: 0


Prediction for image number 3 is: 9


Prediction for image number 4 is: 9


Prediction for image number 5 is: 3


Prediction for image number 6 is: 7


Prediction for image number 7 is: 0


Prediction for image number 8 is: 3


Prediction for image number 9 is: 0


** Native ONNX Support**

PyTorch includes native <a href="http://onnx.ai/">Open Neural Network Exchange (ONNX)</a> support. 

The following code will export models in the standard ONNX
Open Neural Network Exchange) format so that the models can be consumed
by ONNX-compatible platforms, runtimes, visualizers, and more.

PyTorch exports the model by running the model through the training path once
and then save the traced model to a file using ONNX format.

**Tip:** You can test the exported ONNX format model by importing and
running it in an ONNX-comptible famework. See
<a href="https://github.com/onnx/tutorials/tree/master/tutorials">ONNX tutorials</a>
for more information. 


In [15]:
# Export the trained model to ONNX
# one black and white 28 x 28 picture will be used as the input to the model
dummy_input = torch.randn(1, 1, 28, 28) 

onnx_model_filename = "mnist.onnx"
torch.onnx.export(mnist_model, dummy_input,onnx_model_filename)

## Citations
Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based learning applied to document recognition." Proceedings of the IEEE, 86(11):2278-2324, November 1998.

## References
* 1. <a href="https://developer.ibm.com/linuxonpower/deep-learning-powerai/">PowerAI</a>.
* 2. <a href="https://pytorch.org/">PyTorch</a>.
* 3. <a href="https://github.com/pytorch/examples/tree/master/mnist">MNIST mode</a> from the official PyTorch examples repository.